In [1]:
#Import libraries
import pandas as pd
import numpy as np
import gensim.downloader as api
from gensim.models.word2vec import Word2Vec
from transformers import pipeline
import warnings
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN

In [2]:
#Read in 'twitter_df'
twitter_df = pd.read_csv("../dataset/twitter_df.csv")

In [3]:
#Check shape
twitter_df.shape

(22153, 8)

In [5]:
#Drop Unnamed: 0 column
twitter_df.drop(['Unnamed: 0'], axis=1, inplace=True)

In [6]:
twitter_df.head()

tweet_id            tweet_posted_on  \
0  1416181616846811137  2021-07-16 23:43:03+00:00   
1  1416181380279635970  2021-07-16 23:42:07+00:00   
2  1416181225979473920  2021-07-16 23:41:30+00:00   
3  1416180635903868934  2021-07-16 23:39:09+00:00   
4  1416180231350759425  2021-07-16 23:37:33+00:00   

                                              tweets  \
0  🏡 #RemoteWork is on the rise &amp; slowly beco...   
1  Opportunity to join a fantastic team at a hi-t...   
2  Good news for #JobSeekers open to #RemoteWork!...   
3  Four Ways to Energize a Post-Pandemic Workforc...   
4  🚑 These are the tools that will save your #Rem...   

                                      cleaned_tweets  \
0  rise slowly becoming normal which mean need re...   
1  opportunity join fantastic team tech fast pace...   
2  good news open this excellent report from remo...   
3          four way energize post pandemic workforce   
4  these tool that will save your google meet too...   

                    user_location user_location_cleaned  polarity  
0                    New York, NY                    US         1  
1               London | New York                    US         1  
2  Sydney | Hong Kong | Singapore                    HK         1  
3                     Chicago, IL                    US         1  
4                      Duluth, GA                    US         0

In [7]:
twitter_df['user_location_cleaned'].value_counts()

US      9346
None    5284
IN      2255
GB      2242
DE       560
        ... 
SK         1
HN         1
GI         1
GP         1
KH         1
Name: user_location_cleaned, Length: 103, dtype: int64

### Word2Vec

In [8]:
corpus = api.load('text8')
model = Word2Vec(corpus)

In [9]:
#Code written by Caroline
def get_avg_vec(text):
    try:
        # make a list of only the words in the document that are in the word2vec vocabulary
        valid_words = [word for word in text.lower().split() if word in model.wv.key_to_index]
        
        if len(valid_words) == 0:
            print();
            return np.zeros(shape=(100,))
        else:
            # return average word vector, for words in the document that exist in the vocab
            return np.mean([model.wv.get_vector(word) for word in valid_words], axis=0)
    except AttributeError:
        return np.zeros(shape=(100,))

In [10]:
avg_vecs = [get_avg_vec(doc) for doc in twitter_df['cleaned_tweets']]

In [11]:
vec_df = pd.DataFrame(avg_vecs)

In [12]:
vec_df.shape, twitter_df.shape

((22153, 100), (22153, 7))

In [13]:
full_df = pd.concat([twitter_df, vec_df], axis=1)

In [14]:
full_df = full_df.reset_index(drop=True)

In [15]:
full_df.head(3)

tweet_id            tweet_posted_on  \
0  1416181616846811137  2021-07-16 23:43:03+00:00   
1  1416181380279635970  2021-07-16 23:42:07+00:00   
2  1416181225979473920  2021-07-16 23:41:30+00:00   

                                              tweets  \
0  🏡 #RemoteWork is on the rise &amp; slowly beco...   
1  Opportunity to join a fantastic team at a hi-t...   
2  Good news for #JobSeekers open to #RemoteWork!...   

                                      cleaned_tweets  \
0  rise slowly becoming normal which mean need re...   
1  opportunity join fantastic team tech fast pace...   
2  good news open this excellent report from remo...   

                    user_location user_location_cleaned  polarity         0  \
0                    New York, NY                    US         1 -0.366511   
1               London | New York                    US         1 -0.387211   
2  Sydney | Hong Kong | Singapore                    HK         1 -0.608523   

          1         2  ...        90        91        92        93        94  \
0  0.066547 -0.012032  ... -0.134987  0.125815  0.012553 -0.024908  0.229517   
1  0.172549  0.040014  ... -0.171617 -0.465732  0.021583 -0.030787  0.464399   
2 -0.127039 -0.130338  ... -0.169229  0.375155  0.471006 -0.146473 -0.019812   

         95        96        97        98        99  
0  0.772759  0.232187  0.158478 -0.271655  1.288791  
1  0.715457  0.417859  0.225182 -0.151503  0.667928  
2  0.915159  0.552780  0.393013  0.338137  1.187990  

[3 rows x 107 columns]

In [16]:
vec_col_names = full_df.columns[8:]

In [17]:
vec_cols = full_df[vec_col_names]

In [18]:
X = vec_cols
sc = StandardScaler()
X_scaled = sc.fit_transform(X)
X_scaled[0:1]

array([[ 0.4360485 ,  0.15054379,  0.22978401, -1.02705133, -0.53777489,
        -0.31727761,  0.94128391,  0.25681323,  0.09255891, -1.11245274,
         0.56475061, -0.25580502,  0.04954726, -0.48167907,  0.10660003,
        -0.52084498,  0.16229769, -0.40134335,  0.07167905,  0.82075667,
         0.18847967,  0.49689648, -0.11746416,  0.22020288, -1.17468133,
        -0.04646247,  0.61409439,  0.63099828, -0.28273385, -0.95541762,
        -0.67556149,  0.6219114 , -0.06722877, -0.22591564,  0.49946466,
         0.76011354,  0.21551787,  0.70589374, -0.53077102, -0.83700565,
        -0.65915156,  0.22772845, -0.09480693,  0.03483273,  1.15074505,
         1.00578387,  0.62939908, -1.05074177,  0.02739189, -0.28345159,
         0.17219831,  0.60474159,  0.61623413, -0.37604447, -0.38755314,
         0.06753014,  0.31295518,  1.3185351 , -0.78556553,  0.90113898,
        -0.78019131, -0.31247801, -1.77872629,  0.02167562, -0.80059402,
         0.27924546, -0.51065044, -0.70072105,  0.3

### DBSCAN

In [19]:
db = DBSCAN(eps=0.9, min_samples=20)
db.fit(X_scaled)

DBSCAN(eps=0.9, min_samples=20)

In [20]:
full_df['cluster_labels'] = db.labels_

In [21]:
full_df['cluster_labels'].value_counts()

-1     19845
 16      175
 34      153
 17      120
 3        97
 33       78
 14       70
 40       66
 18       61
 46       59
 11       57
 39       53
 37       53
 43       49
 47       48
 51       46
 10       45
 0        44
 5        43
 42       42
 41       41
 4        38
 8        37
 24       36
 20       36
 50       35
 45       34
 22       33
 32       33
 30       32
 23       31
 19       31
 31       30
 38       30
 35       29
 44       29
 48       29
 13       28
 27       28
 6        27
 21       27
 12       27
 29       26
 25       25
 49       24
 26       23
 36       22
 7        22
 2        22
 1        22
 15       21
 28       21
 9        20
Name: cluster_labels, dtype: int64

In [22]:
full_df[full_df['cluster_labels'] == 45]

tweet_id            tweet_posted_on  \
13991  1416696529390391303  2021-07-18 09:49:08+00:00   
14001  1416683094787039234  2021-07-18 08:55:45+00:00   
14004  1416680503231582209  2021-07-18 08:45:27+00:00   
14010  1416677157825351684  2021-07-18 08:32:09+00:00   
14048  1416653011804372992  2021-07-18 06:56:12+00:00   
14049  1416650261494382595  2021-07-18 06:45:17+00:00   
14163  1416531592168853505  2021-07-17 22:53:44+00:00   
14167  1416529699480633345  2021-07-17 22:46:12+00:00   
14169  1416529482710663179  2021-07-17 22:45:21+00:00   
15297  1414450023052554241  2021-07-12 05:02:19+00:00   
15304  1414445727112171521  2021-07-12 04:45:15+00:00   
15312  1414430694864871425  2021-07-12 03:45:31+00:00   
15335  1414415954289725442  2021-07-12 02:46:56+00:00   
15337  1414415534964228100  2021-07-12 02:45:16+00:00   
15376  1414383465127194624  2021-07-12 00:37:50+00:00   
15377  1414382800640397313  2021-07-12 00:35:12+00:00   
15385  1414378591538843648  2021-07-12 00:18:28+00:00   
15404  1414359494390259716  2021-07-11 23:02:35+00:00   
15411  1414354725953150981  2021-07-11 22:43:38+00:00   
15413  1414353559869607944  2021-07-11 22:39:00+00:00   
15416  1414352601458561028  2021-07-11 22:35:12+00:00   
15458  1414335977020182534  2021-07-11 21:29:08+00:00   
15470  1414331719981928451  2021-07-11 21:12:13+00:00   
15486  1414327109296787457  2021-07-11 20:53:54+00:00   
15489  1414324921094840326  2021-07-11 20:45:12+00:00   
15536  1414299005677314048  2021-07-11 19:02:14+00:00   
15543  1414296876426399748  2021-07-11 18:53:46+00:00   
15544  1414296326183997444  2021-07-11 18:51:35+00:00   
15546  1414295342988804102  2021-07-11 18:47:40+00:00   
15549  1414294756503531524  2021-07-11 18:45:20+00:00   
15618  1414269863628058625  2021-07-11 17:06:26+00:00   
15623  1414267339395276810  2021-07-11 16:56:24+00:00   
15626  1414266682009395201  2021-07-11 16:53:47+00:00   
15628  1414265127088570368  2021-07-11 16:47:36+00:00   

                                                  tweets  \
13991  RT @stratorob: Success is not how high you hav...   
14001  RT @stratorob: Success is not how high you hav...   
14004  Success is not how high you have climbed, but ...   
14010  RT @stratorob: Success is not how high you hav...   
14048  RT @stratorob: Success is not how high you hav...   
14049  Success is not how high you have climbed, but ...   
14163  RT @stratorob: Success is not how high you hav...   
14167  RT @stratorob: Success is not how high you hav...   
14169  Success is not how high you have climbed, but ...   
15297  RT @stratorob: Success is not how high you hav...   
15304  Success is not how high you have climbed, but ...   
15312  RT @stratorob: Success is not how high you hav...   
15335  RT @stratorob: Success is not how high you hav...   
15337  Success is not how high you have climbed, but ...   
15376  RT @stratorob: Success is not how high you hav...   
15377  Success is not how high you have climbed, but ...   
15385  RT @stratorob: Success is not how high you hav...   
15404  RT @stratorob: Success is not how high you hav...   
15411  RT @stratorob: Success is not how high you hav...   
15413  RT @stratorob: Success is not how high you hav...   
15416  Success is not how high you have climbed, but ...   
15458  RT @stratorob: Success is not how high you hav...   
15470  RT @stratorob: Success is not how high you hav...   
15486  RT @stratorob: Success is not how high you hav...   
15489  Success is not how high you have climbed, but ...   
15536  RT @stratorob: Success is not how high you hav...   
15543  RT @stratorob: Success is not how high you hav...   
15544  RT @stratorob: Success is not how high you hav...   
15546  RT @stratorob: Success is not how high you hav...   
15549  Success is not how high you have climbed, but ...   
15618  RT @stratorob: Success is not how high you hav...   
15623  RT @stratorob: Success is not how high you hav...   
15626  RT @stratorob: Success is not how h

In [23]:
full_df['tweets'].loc[22137]

'Workplace Flexibility in a Post-COVID World - https://t.co/3PX749AhaJ (via @WF_Institute) #RemoteWork #HR https://t.co/WuZ8rD3CdK'

### k-means

In [24]:
from sklearn.cluster import KMeans
km = KMeans(n_clusters=25)
km.fit(X_scaled)

KMeans(n_clusters=25)

In [25]:
km.cluster_centers_

array([[ 0.18741546,  0.14028999,  0.48671516, ..., -0.61167897,
        -0.21875977, -0.00556535],
       [-0.18426562, -0.79683273, -0.73211998, ...,  0.14526671,
         0.01188764,  0.21234578],
       [ 0.55224105,  0.8059345 ,  1.33368483, ...,  0.10100481,
         0.22900602, -0.25194267],
       ...,
       [-0.88423144,  1.0161835 ,  0.90407901, ...,  0.47258249,
        -1.0294113 ,  0.4543533 ],
       [-0.86954562, -0.90739232, -1.09949364, ...,  0.09064102,
        -0.23044143, -0.73410661],
       [-0.0670875 , -0.14582176, -0.22304136, ..., -0.10677175,
        -0.43820131, -0.16804584]])

In [26]:
full_df['km']=km.labels_

In [27]:
full_df['km'].value_counts()

18    2089
24    2054
4     1524
3     1415
6     1402
0     1388
17    1143
21    1132
19    1059
7     1058
9      979
12     949
2      918
13     795
15     746
1      693
11     626
23     576
16     478
8      400
22     305
20     278
14      98
10      32
5       16
Name: km, dtype: int64

In [28]:
full_df[full_df['km'] == 0]

tweet_id            tweet_posted_on  \
1      1416181380279635970  2021-07-16 23:42:07+00:00   
2      1416181225979473920  2021-07-16 23:41:30+00:00   
40     1416167772875366404  2021-07-16 22:48:02+00:00   
44     1416167769930883073  2021-07-16 22:48:02+00:00   
80     1416154743504719876  2021-07-16 21:56:16+00:00   
...                    ...                        ...   
22129  1418970994862174216  2021-07-24 16:27:03+00:00   
22130  1418970972523221009  2021-07-24 16:26:57+00:00   
22135  1418968495216340999  2021-07-24 16:17:07+00:00   
22136  1418968476429996037  2021-07-24 16:17:02+00:00   
22137  1418967486565949448  2021-07-24 16:13:06+00:00   

                                                  tweets  \
1      Opportunity to join a fantastic team at a hi-t...   
2      Good news for #JobSeekers open to #RemoteWork!...   
40     👀👋 Is this you? @toptalllc is hiring for a Dat...   
44     👀👋 Is this you? @toptalllc is hiring for a Rea...   
80     🚨 .@HRExecMag's @BenefitsConf is currently see...   
...                                                  ...   
22129  Workplace Flexibility in a Post-COVID World - ...   
22130  The New Workplace Paradigm: Taking a High Toll...   
22135  RT @hrbartender: Workplace Flexibility in a Po...   
22136  Workplace Flexibility in a Post-COVID World - ...   
22137  Workplace Flexibility in a Post-COVID World - ...   

                                          cleaned_tweets  \
1      opportunity join fantastic team tech fast pace...   
2      good news open this excellent report from remo...   
40                   this hiring data engineer worldwide   
44          this hiring react native developer worldwide   
80     currently seeking speaker proposal next year e...   
...                                                  ...   
22129             workplace flexibility post covid world   
22130  workplace paradigm taking high toll worker eco...   
22135             workplace flexibility post covid world   
22136             workplace flexibility post covid world   
22137             workplace flexibility post covid world   

                        user_location user_location_cleaned  polarity  \
1                   London | New York                    US         1   
2      Sydney | Hong Kong | Singapore                    HK         1   
40                             Global                  None         1   
44                             Global                  None         1   
80                         Denver, CO                    US         1   
...                               ...                   ...       ...   
22129                   South Florida                    US         1   
22130               State College, PA                    US         0   
22135                 Berlin, Germany                    DE         1   
22136                 Gainesville, FL                    US         1   
22137                   south florida                    US         1   

              0         1         2  ...        92        93        94  \
1     -0.387211  0.172549  0.040014  ...  0.021583 -0.030787  0.464399   
2     -0.608523 -0.127039 -0.130338  ...  0.471006 -0.146473 -0.019812   
40    -0.382177 -0.688483 -0.096223  ...  0.569183  0.346180  0.578697   
44    -0.356146 -0.278961 -0.001065  ...  0.551312  0.323737  0.779059   
80    -0.099488 -0.176510  0.122632  ... -0.245328  0.209891 -0.174113   
...         ...       ...       ...  ...       ...       ...       ...   
22129 -0.566998 -0.232238  0.398531  ... -0.835927 -0.114315  0.381887   
22130 -1.052118 -0.142085  0.216530  ... -0.298820 -0.305085  0.163034   
22135 -0.566998 -0.232238  0.398531  ... -0.835927 -0.114315  0.381887   
22136 -0.566998 -0.232238  0.398531  ... -0.835927 -0.114315  0.381887   
22137 -0.566998 -0.232238  0.398531  ... -0.835927 -0.114315  0.381887   

             95        96        97        98        99  cluster_labels  km  
1      0.715457  0.417859  0.225182 -0.151503  

In [29]:
from sklearn.metrics import silhouette_score
sil_score = silhouette_score(X_scaled, km.labels_)
sil_score

0.030396481633765574